<a href="https://colab.research.google.com/github/Midas0901/Poom/blob/main/ToS_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
import shutil
import sys

folder_path = '/content/'

# Delete all files and folders
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print(f'Failed to delete {file_path}. Reason: {e}')

# Delete this script itself
try:
    os.unlink(sys.argv[0])
except Exception:
    pass

print("All files, folders, and this script have been deleted.")


All files, folders, and this script have been deleted.


In [5]:
# Cell 2
import os
import torch
import numpy as np
import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cpu


In [6]:
# Cell 3
from datasets import load_dataset

# If you need to login with huggingface-cli, do it first (not required if dataset is public)
# !huggingface-cli login

ds = load_dataset("MeeraR/legal-qa-dataset")
dataset = ds["train"].select(range(1200))
print("Number of samples loaded:", len(dataset))
dataset[0]


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


(…)_legal_qa_dataset_unique_questions.jsonl: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/2375 [00:00<?, ? examples/s]

Number of samples loaded: 1200


{'question': 'Explain the rights granted by Section 12 of the Indian Penal Code.',
 'answer': 'This section ensures legal aid for individuals against sexual harassment.',
 'source': 'synthetic_rule_llm',
 'metadata': {'topic': 'IPC',
  'question_type': 'definition',
  'difficulty': 'medium',
  'jurisdiction': 'India'},
 'Answer': None}

In [7]:
# Cell 4
print("Columns in dataset:", dataset.column_names)

# Assume the text field is "question" (change this if your dataset uses a different field)
sample_text = dataset[0].get("question", None)

print("Example text:", sample_text)


Columns in dataset: ['question', 'answer', 'source', 'metadata', 'Answer']
Example text: Explain the rights granted by Section 12 of the Indian Penal Code.


In [8]:
# Cell 5
# Note: If your dataset has real labels, replace this section with the actual label column
labels_map = {
    "Forced Monthly Payments": 0,
    "Data Collection & Sharing with third parties": 1,
    "Unfair Control Over Your Content": 2,
    "No Refund Policy": 3
}

# This example creates rotating placeholder labels for demonstration
df = pd.DataFrame({
    "text": [item.get("question", "") for item in dataset],
    "label": [i % len(labels_map) for i in range(len(dataset))]
})

print(df.shape)
df.head()


(1200, 2)


,text,label
0,Explain the rights granted by Section 12 of th...,0
1,What is the punishment for defamation under In...,1
2,Who can file a complaint under Indian Penal Co...,2
3,Can you summarize Maneka Gandhi vs Union of In...,3
4,Can you summarize Kesavananda Bharati vs State...,0


In [9]:
# Cell 6
from sklearn.model_selection import train_test_split

train_texts, test_texts, train_labels, test_labels = train_test_split(
    df["text"].values, df["label"].values, test_size=0.2, random_state=42, stratify=df["label"].values
)

print("Train size:", len(train_texts), "Test size:", len(test_texts))


Train size: 960 Test size: 240


In [10]:
# Cell 7
from transformers import AutoTokenizer

tokenizer_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

def encode_batch(texts, max_length=128):
    return tokenizer(
        list(texts),
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )

train_enc = encode_batch(train_texts)
test_enc = encode_batch(test_texts)

# ตรวจสอบ shape
print("train input_ids shape:", train_enc["input_ids"].shape)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

train input_ids shape: torch.Size([960, 31])


In [11]:
# Cell 8
import torch
from torch.utils.data import Dataset, DataLoader

class ToSDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {k: v[idx] for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(int(self.labels[idx]), dtype=torch.long)
        return item

batch_size = 16

train_dataset = ToSDataset(train_enc, train_labels)
test_dataset = ToSDataset(test_enc, test_labels)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print("DataLoaders are ready")


DataLoaders are ready


In [12]:
# Cell 9
import torch.nn as nn

class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, hidden_dim=128, num_labels=4):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embed_dim, padding_idx=tokenizer.pad_token_id)
        self.lstm = nn.LSTM(input_size=embed_dim, hidden_size=hidden_dim, batch_first=True)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(hidden_dim, num_labels)

    def forward(self, input_ids, attention_mask=None):
        # input_ids: (batch, seq_len)
        x = self.embedding(input_ids)                 # (batch, seq_len, embed_dim)
        # (optionally) mask paddings by zeroing embeddings where attention_mask == 0
        if attention_mask is not None:
            mask = attention_mask.unsqueeze(-1).float()
            x = x * mask
        output, (h_n, c_n) = self.lstm(x)            # h_n: (num_layers * num_dirs, batch, hidden_dim)
        h_last = h_n[-1]                             # (batch, hidden_dim)
        h_last = self.dropout(h_last)
        logits = self.fc(h_last)                     # (batch, num_labels)
        return logits

vocab_size = tokenizer.vocab_size if hasattr(tokenizer, "vocab_size") else len(tokenizer)
model = LSTMClassifier(vocab_size=vocab_size, num_labels=len(labels_map)).to(device)
print(model)


LSTMClassifier(
  (embedding): Embedding(30522, 128, padding_idx=0)
  (lstm): LSTM(128, 128, batch_first=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc): Linear(in_features=128, out_features=4, bias=True)
)


In [13]:
# Cell 10
import torch.optim as optim
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    all_preds = []
    all_labels = []
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)
    for batch in loop:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch.get("attention_mask", None)
        if attention_mask is not None:
            attention_mask = attention_mask.to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()
        logits = model(input_ids, attention_mask=attention_mask)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = torch.argmax(logits, dim=1).detach().cpu().numpy()
        all_preds.extend(preds.tolist())
        all_labels.extend(labels.detach().cpu().numpy().tolist())

        loop.set_postfix(loss=total_loss / (len(all_preds) / batch_size))

    train_acc = accuracy_score(all_labels, all_preds)
    print(f"Epoch {epoch+1} => loss: {total_loss:.4f}, acc: {train_acc:.4f}")


Epoch 1/3:   0%|          | 0/60 [00:00<?, ?it/s]

Epoch 1 => loss: 83.3458, acc: 0.2417


Epoch 2/3:   0%|          | 0/60 [00:00<?, ?it/s]

Epoch 2 => loss: 83.1841, acc: 0.2458


Epoch 3/3:   0%|          | 0/60 [00:00<?, ?it/s]

Epoch 3 => loss: 83.0445, acc: 0.2562


In [14]:
# Cell 11
from sklearn.metrics import classification_report

model.eval()
preds = []
trues = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Evaluating", leave=False):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch.get("attention_mask", None)
        if attention_mask is not None:
            attention_mask = attention_mask.to(device)
        labels = batch["labels"].to(device)

        logits = model(input_ids, attention_mask=attention_mask)
        batch_preds = torch.argmax(logits, dim=1).cpu().numpy().tolist()
        preds.extend(batch_preds)
        trues.extend(labels.cpu().numpy().tolist())

print(classification_report(trues, preds, target_names=list(labels_map.keys())))


Evaluating:   0%|          | 0/15 [00:00<?, ?it/s]

                                              precision    recall  f1-score   support

                     Forced Monthly Payments       0.00      0.00      0.00        60
Data Collection & Sharing with third parties       0.28      0.20      0.23        60
            Unfair Control Over Your Content       0.26      0.83      0.40        60
                            No Refund Policy       0.00      0.00      0.00        60

                                    accuracy                           0.26       240
                                   macro avg       0.13      0.26      0.16       240
                                weighted avg       0.13      0.26      0.16       240



In [15]:
# Cell 12
def classify_tos(text):
    model.eval()
    enc = tokenizer([text], padding=True, truncation=True, max_length=128, return_tensors="pt")
    input_ids = enc["input_ids"].to(device)
    attention_mask = enc.get("attention_mask", None)
    if attention_mask is not None:
        attention_mask = attention_mask.to(device)
    with torch.no_grad():
        logits = model(input_ids, attention_mask=attention_mask)
        pred = int(torch.argmax(logits, dim=1).cpu().item())
    return list(labels_map.keys())[pred]

# ตัวอย่าง
print(classify_tos("We may sell your personal information"))
print(classify_tos("You must pay every month even if you don't use the service"))


Data Collection & Sharing with third parties
Data Collection & Sharing with third parties


In [ ]:
# Cell: Fast Interactive Model Testing

model.eval()  # call eval() only one time
import torch

print("Start testing the model (type 'exit' to quit)\n")

with torch.no_grad():  # wrap the loop to reduce overhead
    while True:
        user_input = input("Enter Terms of Service text:\n")
        if user_input.lower() == "exit":
            print("Exited testing.")
            break

        # encode only once
        enc = tokenizer([user_input], padding=True, truncation=True, max_length=128, return_tensors="pt")
        input_ids = enc["input_ids"].to(device)
        attention_mask = enc.get("attention_mask", None)
        if attention_mask is not None:
            attention_mask = attention_mask.to(device)

        logits = model(input_ids, attention_mask=attention_mask)
        pred = int(torch.argmax(logits, dim=1).cpu().item())
        predicted_label = list(labels_map.keys())[pred]

        print(f"Model predicted ToS category: {predicted_label}\n")


In [ ]:
import json
from google.colab import files

# Step 1: Upload your notebook
uploaded = files.upload()  # choose your .ipynb file

for filename in uploaded.keys():
    input_file = filename
    output_file = "fixed_" + filename

    # Load notebook
    with open(input_file, "r", encoding="utf-8") as f:
        nb = json.load(f)

    # Remove 'widgets' from notebook metadata if exists
    if "widgets" in nb.get("metadata", {}):
        del nb["metadata"]["widgets"]

    # Remove 'widgets' from each cell metadata if exists
    for cell in nb.get("cells", []):
        if "metadata" in cell and "widgets" in cell["metadata"]:
            del cell["metadata"]["widgets"]

    # Save cleaned notebook
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(nb, f, indent=2)

    # Download the fixed notebook
    files.download(output_file)
    print(f"Fixed notebook saved and downloaded as: {output_file}")
